# Traffic sign UMAP

##  0 . Import

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [ ]:
from torchvision import transforms
from torchvision.models import resnet18, resnet34

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
print(torch.__version__)

In [ ]:
import sys

print("python version is " + sys.version)

In [ ]:
import copy
import os
import pickle
from os.path import join
from typing import Any, List, Tuple

from torchvision.datasets.utils import list_dir, list_files

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import os

In [ ]:
os.chdir("..")

In [ ]:
from src.few_shot_learning.standard_net import StandardNet, StandardNetAdaptater

In [ ]:
from collections import namedtuple

In [ ]:
from src.utils_plot import img_from_tensor, imshow, plot_list

In [ ]:
from src.datasource import get_data_6_rare

In [ ]:
import random

In [ ]:
# import umap
# import umap.plot

In [ ]:
import colorcet as cc
import datashader as ds
import datashader.transfer_functions as tf

### Device selection gpu or not

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
if device == torch.device("cuda"):
    print(torch.cuda.get_device_name(device))

## 1 . Dataset

In [ ]:
path_to_data= "/staging/thesis_data_search/data"

In [ ]:
class_to_search_on,init_data = get_data_6_rare(path_to_data,10,50)

In [ ]:
train_dataset, eval_dataset, test_dataset = init_data()

In [ ]:
len(train_dataset), len(eval_dataset),len(test_dataset)

In [ ]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(
    train_dataset, shuffle=True, num_workers=4, batch_size=batch_size
)

val_loader = torch.utils.data.DataLoader(
    eval_dataset, shuffle=True, batch_size=batch_size, num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, shuffle=True, batch_size=batch_size, num_workers=4
)

##  2. Use a pretrain model

In [ ]:
model = StandardNet(
    len(train_dataset.classes) + len(test_dataset.classes), pretrained=True
)

model = model.to(device)

In [ ]:
model = torch.load("data/new_checkpoint.pt")

In [ ]:
#model

### Umap

In [ ]:
feature_extractor = model

In [ ]:
feature_extractor.resnet.fc = torch.nn.Identity()

In [ ]:
classes = train_dataset.classes

In [ ]:
def get_features_and_label(dataloader,feature_extractor):
    features = []
    labels = []

    with torch.no_grad():

        """
        for class_ in tqdm(classes):
            sampler = ClassSampler(train_dataset, class_, batch_size)

            loader = torch.utils.data.DataLoader(
                train_dataset, batch_sampler=sampler, num_workers=20
        )
        """

        for idx, batch in enumerate(tqdm(dataloader, disable=False)):

            batch_features = feature_extractor(batch["img"].to(device))
            features.append(batch_features)
            labels.append(batch["label"].to(device))

        features = torch.cat(features)
        labels = torch.cat(labels)

        features_np = features.to("cpu").numpy()
        labels_np = labels.to("cpu").numpy()

    return features_np, labels_np

In [ ]:
features_train, labels_train = get_features_and_label(train_loader,feature_extractor)

In [ ]:
features_val, labels_val = get_features_and_label(val_loader,feature_extractor)

In [ ]:
#features_test, labels_test = get_features_and_label(test_loader,feature_extractor)

In [ ]:
# np.savez_compressed("features_labels", features_train=features_train, features_val=features_val,
#                    labels_train=labels_train, labels_val=labels_val)
# a = np.load("features_labels.npz")
# features_train = a['features_train']
# labels_train = a['labels_train']
# features_val = a['features_val']
# labels_val = a['labels_val']

In [ ]:
features_val.shape

In [ ]:
# class_ = 276
class_ = 255

In [ ]:
# class_ += 1
# print(class_)

In [ ]:
class_train_features = np.array([f for f, l in zip(features_train, labels_train) if l == class_])
class_val_features = np.array([f for f, l in zip(features_val, labels_val) if l == class_])
other_train_features = np.array([f for f, l in zip(features_train, labels_train) if l != class_])
other_val_features = np.array([f for f, l in zip(features_val, labels_val) if l != class_])
print(class_train_features.shape, class_val_features.shape)
import fbpca  # Scikit-learn doesn't support uncentered PCA, so I installed this package
(U, s, Va) = fbpca.pca(class_train_features, 2, True)
U, Va = -U, -Va  # The minus is arbitrary, but happens to give positive numbers
print([x.shape for x in (U, s, Va)])

In [ ]:
class_train_features_pca = class_train_features @ Va.T
other_train_features_pca = other_train_features @ Va.T
class_val_features_pca = class_val_features @ Va.T
other_val_features_pca = other_val_features @ Va.T
# plt.plot(train_features_pca[:,0], train_features_pca[:,1], 'go')
# plt.plot(class_train_features_pca[:,0], class_train_features_pca[:,1], 'bx')
plt.plot(other_val_features_pca[:,0], other_val_features_pca[:,1], 'go')
plt.plot(class_val_features_pca[:,0], class_val_features_pca[:,1], 'bx')
# plt.plot(val_features_pca[:,0], val_features_pca[:,0]*0, 'go')
# plt.plot(class_val_features_pca[:,0], class_val_features_pca[:,0]* 0, 'bx')